# YouTube Comments Sentiment Analysis 
Andie's Version <br>
Spring 2018


## 1 Set Up

### 1.1 Import Basic Modules

In [59]:
# Basics
import pandas as pd
import os
import csv
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

### 1.2 Read In Data

In [60]:
os.chdir('/Users/andiedonovan/myProjects/Youtube_Python_Project/AndiesBranch/data/') # change directory
okgo = pd.read_csv('OKGOcomments.csv', delimiter=";", skiprows=2, encoding='latin-1', engine='python') # read in the data

In [61]:
blogs = pd.read_csv('Kagel_social_media_blogs.csv', delimiter="@@@", skiprows=2, encoding='latin-1', engine='python') # read in the data
tweets = pd.read_csv('full-corpus.csv', delimiter=",", skiprows=2, encoding='latin-1', engine='python') # read in the data

In [62]:
okgo.head(3)

,1,Ive been waiting for this music video ever since the song came out on Hungry Ghosts,Unnamed: 2,Unnamed: 3
0,-1.0,Everyone knows brand's papers from.\nBut -No o...,NaN,NaN
1,0.0,ÒYour paper cut balance is: \n-£25279102771Ó,NaN,NaN
2,1.0,OH SHIT WHEN I SAW THIS ON MY FRONT PAGE.........,NaN,NaN


In [63]:
tweets = tweets.drop(['Topic', 'TweetId', "TweetDate"], axis = 1).dropna()
tweets.columns = ["label", "comment"]
tweets.head(3)

,label,comment
0,positive,Now all @Apple has to do is get swype on the i...
1,positive,@Apple will be adding more carrier support to ...
2,positive,Hilarious @youtube video - guy does a duet wit...


In [64]:
tweets.label = tweets.label.replace({'positive': '1.0', 'negative':'-1.0', 'neutral': '0.0', 'irrelevant': '0.0'}, regex=True)

In [65]:
tweets['label'] = pd.to_numeric(tweets['label'], errors='coerce')

In [66]:
tweets.head(2)

,label,comment
0,1.0,Now all @Apple has to do is get swype on the i...
1,1.0,@Apple will be adding more carrier support to ...


In [67]:
blogs.columns = ["label", "comment"]
blogs.head(10)

,label,comment
0,1.0,this was the first clive cussler i've ever re...
1,1.0,i liked the Da Vinci Code a lot
2,1.0,i liked the Da Vinci Code a lot
3,1.0,I liked the Da Vinci Code but it ultimatly di...
4,1.0,that's not even an exaggeration ) and at midn...
5,1.0,I loved the Da Vinci Code but now I want some...
6,1.0,i thought da vinci code was great same with k...
7,1.0,The Da Vinci Code is actually a good movie..
8,1.0,I thought the Da Vinci Code was a pretty good...
9,1.0,The Da Vinci Code is one of the most beautifu...


In [68]:
blogs['label'] = pd.to_numeric(blogs['label'], errors='coerce')

### 1.3 Clean Data Columns

In [69]:
okgo.columns = [
  'label',
  'comment','a','b'
]
okgo = okgo.drop(['a', 'b'], axis = 1).dropna() # drop columns 3 and 4 and missing values

In [70]:
df = pd.concat([okgo, blogs, tweets], ignore_index=False)

In [71]:
df.sample(10)

,label,comment
4055,0.0,@JakeDizzleWins We Need To Speak Sum Real Shit...
1553,-1.0,They should spend the money getting help in wr...
6707,0.0,she helped me bobbypin my insanely cool hat ...
1879,0.0,That was good but when is it coming out? What ...
1616,0.0,#google's galaxy #nexus just came out with #an...
1141,0.0,Today very cheap for 2198597 Icemaker.And Free...
165,1.0,Amazing!!
6596,0.0,Then snuck into Brokeback Mountain which is t...
2514,1.0,Maybe #apple could use some of it's billions t...
81,1.0,I loved the Da Vinci code but it raises a lot...


### 1.4 Remove Non-Alphabetic Characters (including numbers)

In [72]:
for row in range(len(df)):
    line = df.iloc[row,1]
    df.iloc[row,1] = re.sub("[^a-zA-Z]", " ", line)

In [73]:
df.head(5)

,label,comment
0,-1.0,Everyone knows brand s papers from But No on...
1,0.0,Your paper cut balance is
2,1.0,OH SHIT WHEN I SAW THIS ON MY FRONT PAGE ...
3,1.0,Blowing my mind yet again
4,0.0,Should have gone with Dunder Mifflin


## 2 Natural Language Processing

### 2.1 Import Packages

In [74]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

### 2.2 Tokenize Words

In [75]:
df['com_token']=df['comment'].str.lower().str.split()

### 2.2 Remove Stop Words

In [76]:
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('english')

df['com_remv']=df['com_token'].apply(lambda x: [y for y in x if y not in sw])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andiedonovan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2.3 Lemmatization and Stemming

In [77]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

df["com_lemma"] = df['com_remv'] \
    .apply(lambda x : [lemmatizer.lemmatize(y) for y in x]) # lemmatization

df['com_stem']=df['com_lemma'] \
    .apply(lambda x : [ps.stem(y) for y in x]) # stemming

## 3 Data Transformations

### 3.1 Split into Training and Test Data

In [78]:
import sklearn # machine learning
from sklearn.model_selection import train_test_split # splitting up data

In [79]:
df["com_stem_str"] = df["com_stem"].apply(', '.join)

In [80]:
X_train, X_test, Y_train, Y_test = train_test_split(
                                    df["com_stem_str"], df["label"], 
                                    test_size=0.25, 
                                    random_state=42)

### 3.2 Check for missing values

In [81]:
print('lengths training variables: ', len(X_train),",", len(Y_train))
print('lengths testing variables: ', len(X_test),",", len(Y_test), '\n')

print('Are there any missing values?', 
      '\n * Training:', pd.isnull(X_train).values.any(), ',', pd.isnull(Y_train).values.any(), 
      '\n * Testing: ', pd.isnull(X_test).values.any(), ",", pd.isnull(Y_test).values.any())


lengths training variables:  10644 , 10644
lengths testing variables:  3548 , 3548 

Are there any missing values? 
 * Training: False , False 
 * Testing:  False , False


### 3.3 Transform Data to Counts 

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [83]:
tfidf = TfidfVectorizer()
xtrain = tfidf.fit_transform(X_train) # transform and fit training data
xtest = tfidf.transform(X_test) # transform test data from fitted transformer

data_transformed = tfidf.fit_transform(df["com_stem_str"]) # transform entire dataset for cross validation

## 4 Machine Learning Models

In [84]:
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn import svm # support vector machine
from sklearn import metrics # for accuracy/ precision
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent

### 4.1 Multinomial Naive Bayes Model

**Fitting the Model:**

In [85]:
mnb = MultinomialNB()
mnb.fit(xtrain, Y_train) # fit the model on the training data word counts and training data lables

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**Model Predictions:** 

In [86]:
mnb_predict = mnb.predict(xtest) # make our y predictions (labels) on the comment test data
mnb_acc = metrics.accuracy_score(Y_test, mnb_predict)
print('We obtained ', round(mnb_acc, 6), '% accuracy for the model')

We obtained  0.849211 % accuracy for the model


**Classification Report**

In [87]:
print(metrics.classification_report(Y_test, mnb_predict))

             precision    recall  f1-score   support

       -1.0       1.00      0.01      0.01       272
        0.0       0.81      0.97      0.88      1993
        1.0       0.93      0.84      0.89      1283

avg / total       0.87      0.85      0.82      3548



**Confusion Matrix**

In [88]:
metrics.confusion_matrix(Y_test, mnb_predict)

array([[   2,  254,   16],
       [   0, 1929,   64],
       [   0,  201, 1082]])

**Cross Validation of Accuracy:**

In [89]:
scores = cross_val_score(mnb, data_transformed, df["label"], cv=5) # 5 fold cross validation
print("Confidence Interval for Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Confidence Interval for Accuracy: 0.72 (+/- 0.18)


### 4.2 Logistic Regression

**Fitting the Model:**

In [90]:
lr = LogisticRegression(solver='sag', max_iter=100, random_state=42, multi_class="multinomial") # set multinomial setting for multiclass data

**Model Predictions:**

In [91]:
lr.fit(xtrain, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [92]:
lr_predict = lr.predict(xtest)
lr_acc = metrics.accuracy_score(Y_test, lr_predict)
print('We obtained ', round(lr_acc, 6), '% accuracy for the logistic regression model')

We obtained  0.875423 % accuracy for the logistic regression model


**Classification Report:**

In [93]:
print(metrics.classification_report(Y_test, lr_predict))

             precision    recall  f1-score   support

       -1.0       0.63      0.26      0.37       272
        0.0       0.85      0.96      0.90      1993
        1.0       0.95      0.88      0.91      1283

avg / total       0.87      0.88      0.86      3548



**Confusion Matrix:**

In [94]:
metrics.confusion_matrix(Y_test, lr_predict)

array([[  71,  191,   10],
       [  30, 1909,   54],
       [  11,  146, 1126]])

**Cross Validation:**

In [95]:
scores = cross_val_score(lr, data_transformed, df["label"], cv=5) # 5 fold cross validation
print("Confidence Interval for Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Confidence Interval for Accuracy: 0.75 (+/- 0.17)


### 4.3 Linear Support Vector Machine

**Fitting the Model & Predictions:**

In [96]:
svm = svm.SVC()
svm.fit(xtrain, Y_train)
svm_predict = svm.predict(xtest)
svm_acc = metrics.accuracy_score(Y_test, svm_predict)
print('We obtained ', round(svm_acc, 6), '% accuracy for the SVM model')

We obtained  0.561725 % accuracy for the SVM model


**Classification Report:**

In [97]:
print(metrics.classification_report(Y_test, mnb_predict))

             precision    recall  f1-score   support

       -1.0       1.00      0.01      0.01       272
        0.0       0.81      0.97      0.88      1993
        1.0       0.93      0.84      0.89      1283

avg / total       0.87      0.85      0.82      3548



**Confusion Matrix:**

In [98]:
metrics.confusion_matrix(Y_test, lr_predict)

array([[  71,  191,   10],
       [  30, 1909,   54],
       [  11,  146, 1126]])

**Cross Validation:**

In [99]:
scores = cross_val_score(svm, data_transformed, df["label"], cv=5) # 5 fold cross validation
print("Confidence Interval for Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Confidence Interval for Accuracy: 0.57 (+/- 0.00)


### 5.1 K-Nearest Neighbor

**Fitting Model & Predictions:**

In [100]:
from sklearn.neighbors import KNeighborsClassifier # k-NN ensemble method

knn = KNeighborsClassifier()
knn.fit(xtrain, Y_train)

knn_predict = knn.predict(xtest)
knn_acc = metrics.accuracy_score(Y_test, knn_predict)
print('We obtained ', round(knn_acc, 6), '% accuracy for the KNN Bagging model')

We obtained  0.8292 % accuracy for the KNN Bagging model


**Cross Validation:**

In [101]:
scores = cross_val_score(knn, data_transformed, df["label"], cv=5) # 5 fold cross validation
print("Confidence Interval for Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Confidence Interval for Accuracy: 0.77 (+/- 0.19)


### 5.3 Random Forest

**Fitting Model & Predictions:**

In [102]:
from sklearn.ensemble import RandomForestClassifier # random forest ensemble method

ranfor = RandomForestClassifier(n_estimators=10, random_state=10)
ranfor = ranfor.fit(xtrain, Y_train)

rf_predict = ranfor.predict(xtest)
rf_acc = metrics.accuracy_score(Y_test, rf_predict)
print('We obtained ', round(rf_acc, 6), '% accuracy for the Random Forest model')

We obtained  0.866967 % accuracy for the Random Forest model


**Cross Validation:**

In [103]:
scores = cross_val_score(ranfor, data_transformed, df["label"], cv=5) # 5 fold cross validation
print("Confidence Interval for Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Confidence Interval for Accuracy: 0.66 (+/- 0.27)


## 6 Data Visualizations

### 6.1 Table of Model Results

In [104]:
myTable = pd.DataFrame(columns=['Naive Bayes','Support Vect Machine','Logistic Regression', 'K-NN', 'Random Forest'],
                   index=["Accuracy"])
myTable['Naive Bayes']=mnb_acc; myTable['Support Vect Machine']=svm_acc; myTable['Logistic Regression']=lr_acc
myTable['K-NN']= knn_acc; myTable['Random Forest']= rf_acc
myTable

,Naive Bayes,Support Vect Machine,Logistic Regression,K-NN,Random Forest
Accuracy,0.849211,0.561725,0.875423,0.8292,0.866967


In [105]:
labels = list(myTable.columns.values)
values = myTable.iloc[0].values
ypos = np.arange(len(labels))

In [106]:
myTable.iloc[0].values

array([0.84921082, 0.56172492, 0.87542277, 0.82919955, 0.86696731])

In [111]:
import matplotlib.pyplot as plt

In [113]:
'''
plt.plot(ypos, values, align='center', alpha=0.5, type='bar')
plt.xticks(ypos, labels); plt.ylabel('% Accuracy')
plt.xlabel('Model Type'); plt.title('Accuracy of Classification Models')
plt.xticks(rotation=90); plt.show()
'''

"\nplt.plot(ypos, values, align='center', alpha=0.5, type='bar')\nplt.xticks(ypos, labels); plt.ylabel('% Accuracy')\nplt.xlabel('Model Type'); plt.title('Accuracy of Classification Models')\nplt.xticks(rotation=90); plt.show()\n"

## 7 Final Remarks

From the five different machine learning models attempted, Linear Support Vector Machine gave the lowest accuracy (45.3%) while Multinomial Logistic Regression gave the highest (65.5%). Upon first glance, 65% accuracy seems fairly low--however, considering that this was a multinomial classification task (Positive, Neutral or Negative), 65% is significantly higher than a random distribution. Furthermore, because the data was from YouTube comments rather than from a formal body of text, the corpus contained a higher frequency of spelling errors, slang, emojis, names, and foreign languages than would otherwise be expected, all of which added noise to the models. Future advancements may focus on better recognizing and processing these attributes common to social media textual data. 